# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [1]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [2]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3129533 sha256=b542345ee54c9ae778ad5492d53448b0aec224dd9fe90f427cf7997b1821523c
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 자모 단위 FASTTEXT

In [4]:
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=73a7436b1968c6209b62e64cbb329d717e4f955428f1781b6de0b20739edf97b
  Stored in directory: /root/.cache/pip/wheels/93/33/b8/bc2256172a415340e34f3c11ef2b0f3f391769000bb74de988
Successfully built hgtk


In [5]:
!pip install konlpy

from konlpy.tag import Mecab
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 45.0 MB/s eta 0:00:00
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 29.57 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-14 23:42:22--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
R

In [3]:
import hgtk
from tqdm import tqdm
from konlpy.tag import Mecab, Okt

def word_to_jamo(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''

    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            #decomposed_token = decomposed_token + cho + jung + jong
            decomposed_token = decomposed_token + cho ## 초성만 넣을 경우의 테스트

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    # 단 어 토 큰 의 자 모 단 위 분 리 결 과 를 추 가
    return decomposed_token


#mecab = Okt()
mecab = Mecab()

def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]


In [ ]:
tokenize_by_jamo("간장 치킨")

['ㄱㅏㄴㅈㅏㅇ', 'ㅊㅣ-ㅋㅣㄴ']

In [4]:
df = pd.read_csv("/gdrive/MyDrive/2023beaver/real_finish.csv")

In [5]:
df = df.dropna()

In [ ]:
df = pd.read_csv("/gdrive/MyDrive/2023beaver/Kor_standard_menu.csv", encoding='cp949')

In [ ]:
df['식품대분류명'].value_counts()

빵 및 과자류         4012
음료 및 차류         2262
유제품류 및 빙과류       363
볶음류              204
국 및 탕류           182
밥류               179
생채·무침류           173
면 및 만두류          166
찌개 및 전골류         152
구이류              120
찜류                94
전·적 및 부침류         77
조림류               75
나물·숙채류            73
튀김류               50
죽 및 스프류           46
김치류               30
장아찌·절임류           18
젓갈류                9
수·조·어·육류           7
장류, 양념류            5
곡류, 서류 제품          3
두류, 견과 및 종실류       2
과일류                1
Name: 식품대분류명, dtype: int64

In [ ]:
df['업체명'].value_counts()

해당없음         1639
지정환피자         432
투썸 플레이스       301
스타벅스          256
요거프레소         248
             ... 
프랭크버거           9
망원동티라미수         8
신세계푸드 피코크       6
CJ 쿡킷           6
코코호도            1
Name: 업체명, Length: 91, dtype: int64

In [ ]:
df = df[['식품명', '식품대분류명', '대표식품명']]

In [ ]:
df.info()

AttributeError: ignored

In [ ]:
df.columns = ['title', 'large_category', 'category']

In [ ]:
df['title'] = df['title'].replace('_', ' ', regex=True)

In [ ]:
df.to_csv("Kor_standard_menu_clean.csv")

hanspell 사용을 위해서 500자까지 채워넣는 방식 검토중


-> hanspell 자체를 사용하지 않는 방향으로 생각

In [ ]:
tmp_str = ''
tmp_max = 500

i = 0
while len(tmp_str) < tmp_max:
  if len(tmp_str) >= tmp_max:
    break
  tmp_str += df['title'][i]
  i += 1

tmp_str

'국내산 배추 김치 포기 김장김치 주문피코크 조선호텔 포기김치대상 종가집 행복이온 포기김치대상 종가집 전라도 행복이온 포기김치선화동매운실비김치 맛있게 매운 선화동 실비 김치엄마손맛 포기김치대상 종가집 우리땅배추김치 태백옥과맛있는김치 김권태김치 포기 배추 김치홍진경더김치 포기김치대상 종가집 포기김치팔공 명품 배추김치안동학가산김치 안동학가산 배추김치풀무원 사계절 아삭 포기김치대상 종가집 전라도 포기김치전라도 해남 김장 포기 김치빅마마 이혜정의 맛있는 포기김치팔공산김치 팔공산 명품포기김치바로푸드 친정김치 배추김치착한김치 맛없다면 무료반품나홀로족세트상품 대전 선화동실비김치국내산 고랭지 포기김치 대상수상 김치김춘자명인 포기 배추 김치 김장김치 주문안동학가산김치 국내산 배추 포기김치 산지직송노브랜드 별미 포기김치안동학가산 고랭지 포기김치총각김치 깍두기 외대복식품 한복선 대복 포기김치종가집 국산 포기김치 태백 남도 종갓집 배추김치 갓담은 김장종가집 김장김치 증정 총각김치제일제당 비비고 포기배추김치'

In [ ]:
tmp_str_blank_del = tmp_str.replace(" ", '')

In [ ]:
tmp_str_blank_del

'국내산배추김치포기김장김치주문피코크조선호텔포기김치대상종가집행복이온포기김치대상종가집전라도행복이온포기김치선화동매운실비김치맛있게매운선화동실비김치엄마손맛포기김치대상종가집우리땅배추김치태백옥과맛있는김치김권태김치포기배추김치홍진경더김치포기김치대상종가집포기김치팔공명품배추김치안동학가산김치안동학가산배추김치풀무원사계절아삭포기김치대상종가집전라도포기김치전라도해남김장포기김치빅마마이혜정의맛있는포기김치팔공산김치팔공산명품포기김치바로푸드친정김치배추김치착한김치맛없다면무료반품나홀로족세트상품대전선화동실비김치국내산고랭지포기김치대상수상김치김춘자명인포기배추김치김장김치주문안동학가산김치국내산배추포기김치산지직송노브랜드별미포기김치안동학가산고랭지포기김치총각김치깍두기외대복식품한복선대복포기김치종가집국산포기김치태백남도종갓집배추김치갓담은김장종가집김장김치증정총각김치제일제당비비고포기배추김치'

In [ ]:
spell_check(tmp_str_blank_del)

'국내산배추김치포기김장김치주문피코크조선호텔포기김치대상종가집행복이온포기김치대상종가집전라도행복이온포기김치선화동매운실비김치맛있게매운선화동실비김치엄마손맛포기김치대상종가집우리땅배추김치태백옥과맛있는김치김권태김치포기배추김치홍진경더김치포기김치대상종가집포기김치팔공명품배추김치안동학가산김치안동학가산배추김치풀무원사계절아삭포기김치대상종가집전라도포기김치전라도해남김장포기김치빅마마이혜정의맛있는포기김치팔공산김치팔공산명품포기김치바로푸드친정김치배추김치착한김치맛없다면무료반품나홀로족세트상품대전선화동실비김치국내산고랭지포기김치대상수상김치김춘자명인포기배추김치김장김치주문안동학가산김치국내산배추포기김치산지직송노브랜드별미포기김치안동학가산고랭지포기김치총각김치깍두기외대복식품한복선대복포기김치종가집국산포기김치태백남도종갓집배추김치갓담은김장종가집김장김치증정총각김치제일제당비비고포기배추김치'

In [ ]:
test_str = "국내산국시"
test_str2 = "국래산국시"

In [ ]:
spell_check(test_str)

'국내산 국시'

In [ ]:
spell_check(test_str2)

'국내산 국시'

국시 -> 국수 로 바꿔줘야 하는데 hanspell은 이런 도메인 특화의 표준화는 하지 못함

In [ ]:
len(df['title'][0])

20

In [ ]:
df['title'][0]

'국내산 배추 김치 포기 김장김치 주문'

In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.10, random_state=2)
train_data, val_data = train_test_split(train_data, test_size=0.15, random_state=2)

In [ ]:
df['sub_category'].value_counts()

차류            15382
커피             8921
가루/분말류         5203
도시락/밥류         3913
해산물/어패류        3571
              ...  
피자/핫도그/햄버거        3
세트                3
겉절이               2
절임배추              1
오이소박이             1
Name: sub_category, Length: 195, dtype: int64

- 데이터 불균형이 심해서 가중치가 제대로 반영이 안되는 현상이 있음

In [ ]:
df['title'][0]

'국내산 배추 김치 포기 김장김치 주문'

In [ ]:
word_to_jamo("바나나우유")

'ㅂㅏ-ㄴㅏ-ㄴㅏ-ㅇㅜ-ㅇㅠ-'

In [ ]:
word_to_jamo("바나나 우유")

'ㅂㅏ-ㄴㅏ-ㄴㅏ- ㅇㅜ-ㅇㅠ-'

In [ ]:
word_to_jamo(df['title'][0])

'ㄱㅜㄱㄴㅐ-ㅅㅏㄴ ㅂㅐ-ㅊㅜ- ㄱㅣㅁㅊㅣ- ㅍㅗ-ㄱㅣ- ㄱㅣㅁㅈㅏㅇㄱㅣㅁㅊㅣ- ㅈㅜ-ㅁㅜㄴ'

In [ ]:
print(tokenize_by_jamo(df['title'][0]))

['ㄱㅜㄱㄴㅐ-ㅅㅏㄴ', '100', '%', 'ㅈㅓㄴㄹㅏ-ㄷㅗ-', 'ㅂㅐ-ㅊㅜ-', 'ㄱㅣㅁㅊㅣ-', 'ㅍㅗ-ㄱㅣ-', 'ㄱㅣㅁㅈㅏㅇ', 'ㄱㅣㅁㅊㅣ-', 'ㅈㅜ-ㅁㅜㄴ']


In [6]:
def tokenized(data):
  tokenized_data=[]

  for sample in tqdm(data['title'].to_list()):
      tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
      tokenized_data.append(tokenzied_sample)

  return tokenized_data

In [7]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

# 1. 초 기 입 력
# jamo_sequence = ' ﾤ ￂ ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '

    while index < len(jamo_sequence):
    # 문 자 가 한 글 ( 정 상 적 인 자 모 ) 이 아 닐 경 우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

    # 문 자 가 정 상 적 인 자 모 라 면 초 성 , 중 성 , 종 성 을 하 나 의 토 큰 으 로 간 주 .
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

# 2. 자 모 단 위 토 큰 화 완 료
# tokenized_jamo : [' ﾤ ￂ ﾱ ', ' ﾧ ￌ ﾷ ', ' ﾵ ￃ ﾷ ']

        word = ''
        try:
            for jamo in tokenized_jamo:
            # 초 성 , 중 성 , 종 성 의 묶 음 으 로 추 정 되 는 경 우
                if len(jamo) == 3:
                    if jamo[2] == "-":
                    # 종 성 이 존 재 하 지 않 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1])
                    else:
                # 종 성 이 존 재 하 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
                # 한 글 이 아 닌 경 우
                else:
                    word = word + jamo

            # 복 원 중 (hgtk.letter.compose) 에 러 발 생 시 초 기 입 력 리 턴 .
            # 복 원 이 불 가 능 한 경 우 예 시 ) ' ﾤ ! ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                return jamo_sequence

        # 3. 단 어 로 복 원 완 료
        # word : ' 남 동 생 '

    return word

# 학습데이터 구축
학습데이터를 만듭니다.

In [9]:
train_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

맞춤법 검사 hanspell 은 2000개에 1시간이 걸린다. 활용할 방법을 찾아야 할듯

In [10]:
train_token = tokenized(train_data)
val_token = tokenized(val_data)
test_token = tokenized(test_data)
df_token = tokenized(df)

100%|██████████| 205379/205379 [00:21<00:00, 9689.51it/s]


In [11]:
with open('train_data.txt', 'w') as out:
    for i in tqdm(range(len(train_token)), unit=' line'):
          out.write("__label__" + train_data['category'][i] + "\t" + ' '.join(train_token[i]) + '\n')

with open('val_data.txt', 'w') as out:
    for i in tqdm(range(len(val_token)), unit=' line'):
        out.write("__label__" + val_data['category'][i] + "\t" + ' '.join(val_token[i]) + '\n')

with open('test_data.txt', 'w') as out:
    for i in tqdm(range(len(test_token)), unit=' line'):
        out.write("__label__" + test_data['category'][i] + "\t" + ' '.join(test_token[i]) + '\n')

100%|██████████| 20538/20538 [00:00<00:00, 156239.16 line/s]


In [ ]:
with open('df_data.txt', 'w') as out:
    for i in tqdm(range(len(df_token)), unit=' line'):
        out.write("__label__" + df['category'][i] + "\t" + ' '.join(df_token[i]) + '\n')

100%|██████████| 8303/8303 [00:00<00:00, 114051.57 line/s]


# Gensim을 통한 학습

In [ ]:
!pip install gensim --upgrade

In [ ]:
!pip install numpy --upgrade

In [ ]:
!pip install -U scikit-learn

In [ ]:
from gensim.models import FastText
from gensim.utils import tokenize
from gensim import utils
from gensim.test.utils import datapath

class MyIter:
    def __iter__(self):
        path = datapath('/content/test_data.txt')
        with utils.open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))

model3 = FastText(vector_size=4, window=3, min_count=1)
model3.build_vocab(corpus_iterable=MyIter())
total_examples = model3.corpus_count
model3.train(corpus_iterable=MyIter(), total_examples=total_examples, epochs=5)

(3276530, 4152075)

### 저장

In [ ]:
model_fname = 'fasttext'

In [ ]:
model3.save(model_fname)
# https://projector.tensorflow.org/ 에서 시각화 하기 위해 모델을 따로 저장
model3.wv.save_word2vec_format(model_fname + "_vis")

In [ ]:
model3 = FastText.load(model_fname)

In [ ]:
def transform_gensim(word_sequence):
    return [(jamo_to_word(word), similarity) for (word, similarity) in word_sequence]

In [ ]:
transform_gensim(model3.wv.most_similar(word_to_jamo("찜닭"), topn=5))

[('맛있', 0.9995765686035156),
 ('냉동', 0.9995282292366028),
 ('냉장육', 0.9994854927062988),
 ('목살', 0.9984344840049744),
 ('통살', 0.9979778528213501)]

In [ ]:
transform_gensim(model3.wv.most_similar(word_to_jamo("볶음밥"), topn=5))

[('덮밥', 0.9965663552284241),
 ('즉', 0.996501088142395),
 ('닭볶음밥', 0.996482253074646),
 ('쌈밥', 0.9961134791374207),
 ('맙', 0.9931130409240723)]

업데이트

In [ ]:
path = datapath('/content/train_data.txt')
old_vector = []
with utils.open(path, 'r', encoding='utf-8') as fin:
    for line in fin:
      old_vector.append(line)

In [ ]:
path = datapath('/content/val_data.txt')
new_vector = []
with utils.open(path, 'r', encoding='utf-8') as fin:
    for line in fin:
      new_vector.append(line)

In [ ]:
import numpy as np
np.allclose(old_vector, new_vector, atol=1e-4)

In [ ]:
model3.build_vocab(new_vector, update=True)
model3.train(new_vector, total_examples=len(new_vector), epochs=model3.epochs)

(6759131, 21611110)

# 학습


In [12]:
import fasttext

model = fasttext.train_supervised(input='train_data.txt', autotuneValidationFile='val_data.txt')

In [13]:
model.predict(word_to_jamo("삼계죽"))

(('__label__전통주',), array([0.99090558]))

In [22]:
model.predict("ㅊㅋ")

(('__label__냉동/간편조리식품',), array([0.9279654]))

In [23]:
model.predict("ㅋㅍ")

(('__label__커피/차류',), array([0.99869269]))

In [25]:
model.predict("ㄱㅊ")

(('__label__김치',), array([1.00001001]))

In [26]:
model.predict("ㅅㄱㅅ")

(('__label__축산물',), array([1.00000036]))

# 테스트
결과는 (샘플, 정확도, 재현율)

In [20]:
model.test("test_data.txt")

(20538, 0.8758399065147532, 0.8758399065147532)

NAVER 데이터셋
- 초성만을 넣은 상태에서도 특정 키워드의 예측은 잘되었다.


- 테스트 정확도가 0.8758 정도로 초성만을 사용하면 정확도가 약간은 내려가지만 분명히 초성이 중요한 특성임을 알려주고 있다.

In [ ]:
model.save_model("fasttext_supervised_basemodel.bin") # 모 델 저 장

In [ ]:
import fasttext
model2 = fasttext.train_unsupervised('df_data.txt', model='skipgram')

In [ ]:
model2.get_nearest_neighbors(word_to_jamo("찜닭"), k=10)

[(0.9999814629554749, 'ㄷㅏㄺㅉㅣㅁ'),
 (0.9999790787696838, 'ㅉㅣㅁ'),
 (0.9999781847000122, 'ㄱㅏㄹㅂㅣ-ㅉㅣㅁ'),
 (0.9999775290489197, 'ㅇㅑㅇㄴㅕㅁ'),
 (0.9999749064445496, 'ㄱㅗ-ㅊㅜ-ㅈㅏㅇ'),
 (0.9999746084213257, 'ㅁㅐ-ㅇㅜㄴ'),
 (0.9999740719795227, 'ㅁㅐ-ㅇㅜㄴㅌㅏㅇ'),
 (0.999971866607666, 'ㅂㅣ-ㅂㅣㅁㅂㅏㅂ'),
 (0.9999716877937317, 'ㄷㅏㄺㅌㅟ-ㄱㅣㅁ'),
 (0.9999716877937317, 'ㄱㅗ-ㅊㅜ-')]

In [ ]:
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

가장 가까운 단어(유사도 기준)

In [ ]:
print(transform(model2.get_nearest_neighbors(word_to_jamo("치킨"), k=10)))

[('흑미', 0.9999756217002869), ('백미', 0.9999728202819824), ('스위티', 0.9999725222587585), ('도이치', 0.9999719262123108), ('가자미', 0.9999687075614929), ('리코타', 0.9999684691429138), ('이탈리안', 0.9999672770500183), ('통밀', 0.999966561794281), ('자바', 0.9999661445617676), ('생강차', 0.9999660849571228)]


유추(의미 기준)

In [ ]:
transform(model2.get_analogies(word_to_jamo("치킨"), word_to_jamo("양념"), word_to_jamo("마늘")))

[('치아', 0.999954104423523),
 ('코코아', 0.999951958656311),
 ('바닐라', 0.9999518990516663),
 ('디아', 0.9999489188194275),
 ('슈페너', 0.9999486804008484),
 ('페페', 0.9999475479125977),
 ('페퍼민트', 0.9999456405639648),
 ('스퀘어', 0.9999456405639648),
 ('페퍼로니', 0.9999446868896484),
 ('루미', 0.9999446272850037)]

# 저장된 모델 DB에 삽입 과정 

BLOB 방식은 안된다. bin을 BLOB로 파싱하는데 무리가 있다

In [ ]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
import pandas as pd
import psycopg2
import numpy
from psycopg2.extensions import register_adapter, AsIs
  
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(numpy.float64, addapt_numpy_float64)
register_adapter(numpy.int64, addapt_numpy_int64)

conn = None
try:
    # connect to the PostgreSQL server
    host = 'arjuna.db.elephantsql.com'
    user = 'kwcuclpe'
    password = ''
    database = 'kwcuclpe'

    conn = psycopg2.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
  
    # Creating a cursor with name cur.
    cur = conn.cursor()
    cur.execute(
        "CREATE TABLE blob_datastore (s_no serial, file_name VARCHAR ( 50 ), blob_data bytea)")
    # SQL query to insert data into the database.
    insert_script = '''
        INSERT INTO blob_datastore(s_no,file_name,blob_data) VALUES (%s,%s,%s);
    '''
  
    # open('File,'rb').read() is used to read the file.
    # where open(File,'rb').read() will return the binary data of the file.
    # psycopg2.Binary(File_in_Bytes) is used to convert the binary data to a BLOB data type.
    BLOB_1 = psycopg2.Binary(
        open('/content/fasttext_supervised_basemodel.bin', 'rb').read())       # Video
    # BLOB_2 = psycopg2.Binary(
    #     open('files\Octa.jpg', 'rb').read())        # Image
    # BLOB_3 = psycopg2.Binary(open('files\Type.gif', 'rb').read())        # GIF
    # BLOB_4 = psycopg2.Binary(open('files\BlobNotes.pdf', 'rb').read())   # PDF
  
    # And Finally we pass the above mentioned values to the insert_script variable.
    insert_values = [(1, 'fasttext_basemodel', BLOB_1)]
  
    # The execute() method with the insert_script & insert_value as argument.
    for insert_value in insert_values:
        cur.execute(insert_script, insert_value)
        print(insert_value[0], insert_value[1],
              "[Binary Data]", "row Inserted Successfully")
  
    # SQL query to fetch data from the database.
    cur.execute('SELECT * FROM BLOB_DataStore')
  
    # open(file,'wb').write() is used to write the binary data to the file.
    for row in cur.fetchall():
        BLOB = row[2]
        open("new"+row[1], 'wb').write(BLOB)
        print(row[0], row[1], "BLOB Data is saved in Current Directory")
  
    # Close the connection
    cur.close()
  
except(Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        
        # Commit the changes to the database
        conn.commit()

이에 따라 워드임베딩만 fastText로 진행하고, 모델은 SVM을 사용하여 피클링을 시도해본다.

## 데이터 준비

In [ ]:
# -*- coding: utf-8 -*-

## 환경설정
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk

In [ ]:
data_df = pd.read_csv("/gdrive/MyDrive/2023beaver/final_clean.csv")

In [ ]:
data_df = pd.read_csv("/content/Kor_standard_menu_clean.csv")

In [ ]:
data_df['sub_category'].value_counts()

차류          20143
커피          11648
가루분말류        7384
도시락밥류        6485
해산물어패류       4448
            ...  
옥수수콩            3
피자핫도그햄버거        3
세트              3
절임배추            1
오이소박이           1
Name: sub_category, Length: 195, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# train_corpus, test_corpus, train_label_names, test_label_names =\
#                                  train_test_split(np.array(data_df['title']), np.array(data_df['sub_category']),
#                                                        test_size=0.33, random_state=42)
train_corpus, test_corpus = train_test_split(data_df, test_size=0.2, random_state=42)
#train_corpus, val_corpus = train_test_split(data_df, test_size=0.15, random_state=42)

feature = 'title'
target = 'category'

train_label_names = train_corpus[target]
#val_label_names = val_corpus[target]
test_label_names = test_corpus[target]

train_corpus = train_corpus['title']
#val_corpus = val_corpus['title']
test_corpus = test_corpus['title']

train_corpus.shape, test_corpus.shape

((6642,), (1661,))

In [ ]:
def tokenized(data):
  tokenized_data=[]

  for sample in tqdm(data):
      tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
      tokenized_data.append(tokenzied_sample)

  return tokenized_data

In [ ]:
tokenized_train = tokenized(train_corpus)
#tokenized_val = tokenized(val_corpus)
tokenized_test = tokenized(test_corpus)

100%|██████████| 1661/1661 [00:00<00:00, 12804.03it/s]


## 학습 준비 및 실행

gensim 4.3.0 기준 , 코랩은 업그레이드 해야함

In [ ]:
!pip install gensim --upgrade

In [ ]:
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.1 which is incompatible.


In [ ]:
from gensim.models.fasttext import FastText

ft_num_features = 400 # 4의 배수에서 좋은 성능이 나온다고 함

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# sg decides whether to use the skip-gram model (1) or CBOW (0)
ft_model = FastText(tokenized_train, vector_size=ft_num_features, window=50, 
                    min_count=1, sample=1e-3, sg=1, workers=10)

In [ ]:
model_fname=f"/gdrive/MyDrive/2023beaver/fastText_{ft_num_features}"

In [ ]:
ft_model.save(model_fname)
# https://projector.tensorflow.org/ 에서 시각화 하기 위해 모델을 따로 저장
ft_model.wv.save_word2vec_format(model_fname + "_vis")

### 이미 저장된 모델 로드

In [ ]:
from gensim.models.fasttext import FastText

ft_model = FastText.load(model_fname)

fastText 워드임베딩 pickle화 시도

-> 가능은 한데, 용량이 1.3G로 상당히 크다. 문제 해결 X

In [ ]:
import pickle
filename = model_fname + ".pickle"
pickle.dump(ft_model, open(filename, "wb"))

joblib을 통한 압축 시도

compress 파라미터가 압축을 의미하며, 높을수록 오래걸리지만 용량이 적어진다.(0 ~ 9까지)

https://joblib.readthedocs.io/en/latest/generated/joblib.dump.html

compress 9 (최대)에서 710M까지 줄여졌다.

-> 여전히 문제 해결X

In [ ]:
import joblib

filename = model_fname + ".joblib"

# save model
joblib.dump(ft_model, filename, compress=9)

['/gdrive/MyDrive/2023beaver/fastText_40.joblib']

In [ ]:
def document_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in tqdm(corpus)]
    return np.array(features)

In [ ]:
# generate averaged word vector features from word2vec model
avg_ft_train_features = document_vectorizer(corpus=tokenized_train, model=ft_model,
                                                     num_features=ft_num_features)
# avg_ft_val_features = document_vectorizer(corpus=tokenized_val, model=ft_model,
#                                                      num_features=ft_num_features)
avg_ft_test_features = document_vectorizer(corpus=tokenized_test, model=ft_model,
                                                    num_features=ft_num_features)

100%|██████████| 1661/1661 [00:00<00:00, 41430.71it/s]


In [ ]:
print('FastText model:> Train features shape:', avg_ft_train_features.shape)
#print(' Val features shape:', avg_ft_val_features.shape)
print(' Test features shape:', avg_ft_test_features.shape)

FastText model:> Train features shape: (6642, 400)
 Test features shape: (1661, 400)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(loss='log', penalty='l2', random_state=42, max_iter=1000, class_weight='balanced', alpha=0.00001)
svm.fit(avg_ft_train_features, train_label_names)
######### CV를 통한 평균 정확도 (너무 오래 걸려서 필요할 때만 시행)
# svm_ft_cv_scores = cross_val_score(svm, avg_ft_train_features, train_label_names, cv=5)
# svm_ft_cv_mean_score = np.mean(svm_ft_cv_scores)
# print('CV Accuracy (5-fold):', svm_ft_cv_scores)
# print('Mean CV Accuracy:', svm_ft_cv_mean_score)
#########
svm_ft_test_score = svm.score(avg_ft_test_features, test_label_names)
print('Test Accuracy:', svm_ft_test_score)

Test Accuracy: 0.555689343768814


fastText vector_size 1000 + svm alpha default=0.0001

- Test Accuracy: 0.7781215414025556

fastText vector_size 1000 + svm alpha 0.00001

- Test Accuracy: 0.7945554360744808

fastText vector_size 1000 + svm alpha 0.00001, log

- Test Accuracy: 0.8117607108495104

fastText vector_size 640 + svm alpha default=0.0001

- Test Accuracy: 0.7758409266700624

fastText vector_size 640 + svm alpha 0.00001

- Test Accuracy: 0.7922538532646631

fastText vector_size 300 + svm alpha 0.01

- Test Accuracy: 0.68043843

fastText vector_size 300 + svm alpha 0.001

- Test Accuracy: 0.7242439677778406

fastText vector_size 300 + svm alpha 0.0001

- Test Accuracy: 0.7700869696455185

fastText vector_size 300 + svm alpha 0.00001

- Test Accuracy: 0.7906691569038051

fastText vector_size 300 + svm alpha 0.000001

- Test Accuracy: 0.7646348595468523

fastText vector_size 40, min_count 100 + svm alpha 0.00001 , log

- Test Accuracy: 0.6987379025411738

parfit을 이용한 SGDClassifier 하이퍼파라미터최적화 (오류로 인해 PASS)

RandomizedSearchCV을 이용한 최적화

In [ ]:
!pip install parfit

In [ ]:
loss = ['hinge', 'log', 'modified_huber', 'squared_hinge',  
'perceptron'] 
penalty = ['l1', 'l2', 'elasticnet'] 
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] 
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive'] 
class_weight = ['balanced']
eta0 = [1, 10, 100] 
param_distributions = dict(loss=loss, 
penalty=penalty, 
alpha=alpha, 
learning_rate=learning_rate, 
class_weight=class_weight, 
eta0=eta0)

fit_params={"early_stopping_rounds":5,
                "eval_metric" : "roc_auc", 
                "eval_set" : [[avg_ft_val_features, val_label_names]]
               }

from sklearn.linear_model import SGDClassifier 
from sklearn.model_selection import RandomizedSearchCV 
sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=5) 
random = RandomizedSearchCV(estimator=sgd, 
param_distributions=param_distributions, 
scoring='roc_auc', 
verbose=1, n_jobs=-1, 
n_iter=5) 
random_result = random.fit(avg_ft_train_features, train_label_names,**fit_params) 
print('Best Score: ', random_result.best_score_) 
print('Best Params: ', random_result.best_params_) 

NameError: ignored

In [ ]:
predict_feature = document_vectorizer(corpus=tokenized(["아메리카노"]), model=ft_model,
                                                     num_features=ft_num_features)

100%|██████████| 1/1 [00:00<00:00, 4529.49it/s]


In [ ]:
svm.predict(predict_feature)

array(['라멘'], dtype='<U11')

In [ ]:
import pickle
filename = "/gdrive/MyDrive/2023beaver/svm_40_alpha00001_log.pickle"
pickle.dump(svm, open(filename, "wb"))

In [ ]:
loaded_model = pickle.load(open(filename, "rb"))

In [ ]:
loaded_model.predict(predict_feature)

array(['튀김류', '맛살/게살'], dtype='<U11')

## tf-idf + sgd 분류

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

data_df = pd.read_csv("/content/Kor_standard_menu_clean.csv")
vect = TfidfVectorizer()
X = vect.fit_transform(data_df['title'])
y = data_df['category']

model = SGDClassifier(loss='log', penalty='l2', random_state=42, max_iter=1000, class_weight='balanced', alpha=0.00001)
model.fit(X, y)

SGDClassifier(alpha=1e-05, class_weight='balanced', loss='log', random_state=42)

In [ ]:
X_pred = vect.transform(['아이스 아메리카노', '돼지 국밥'])
y_pred = model.predict(X_pred)

print(y_pred)

['커피' '국밥']
